In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Load Packages
import jax.numpy as jnp
from plotly.subplots import make_subplots

# Load Custom Modules
from plots import plot_true_function, add_optimization_path
from core import optimize_function, initial_point

In [36]:
# Define Constants
X_MIN, X_MAX, X_POINTS = -10, 10, 100
Y_MIN, Y_MAX, Y_POINTS = -10, 10, 100

x = jnp.linspace(X_MIN, X_MAX, X_POINTS)
y = jnp.linspace(Y_MIN, Y_MAX, Y_POINTS)

# Randomly sample initial point
pi = initial_point(42, minval=-10, maxval=10)

# Set Seeds for generating random samples during optimization
seeds = [42, 42, 42, 42]

# Set Noise Parameters
true_params = (0, 0, 1)
noise_params = (5, 5, 1)

# Set Gradient Agreement Filter
gaf_parameters = {
    'method': 'pairwise',
    'threshold': 1 - jnp.cos(jnp.pi/4), # 0.97
}

In [37]:
# Define function we want to optimize
def cone(x, params):
    return params[2]*jnp.linalg.norm(x - jnp.array([params[0], params[1]]), axis=-1)

In [38]:
cfig = make_subplots(rows=2, cols=2, subplot_titles=['SGD', 'Adam', 'AdaGrad', 'LAMB'])

# SGD
p_sgd = optimize_function(cone, pi, true_params, noise_params, steps=50, learning_rate=0.25, batch_size=3, method='sgd', seed=seeds[0])
fig = plot_true_function(x, y, lambda x: cone(x, true_params))
fig.update_layout(title=f'SGD', title_x=0.5)
add_optimization_path(fig, p_sgd)
cfig.add_trace(fig.data[0], row=1, col=1)
cfig.add_trace(fig.data[1], row=1, col=1)

# Adam
p_adam = optimize_function(cone, pi, true_params, noise_params, steps=50, learning_rate=0.2, batch_size=10, method='adam', seed=seeds[1])
fig = plot_true_function(x, y, lambda x: cone(x, true_params))
fig.update_layout(title=f'Adam', title_x=0.5)
add_optimization_path(fig, p_adam)
cfig.add_trace(fig.data[0], row=1, col=2)
cfig.add_trace(fig.data[1], row=1, col=2)

# AdaGrad
p_adagrad = optimize_function(cone, pi, true_params, noise_params, steps=50, learning_rate=1.0, batch_size=10, method='adagrad', seed=seeds[2])
fig = plot_true_function(x, y, lambda x: cone(x, true_params))
fig.update_layout(title=f'AdaGrad', title_x=0.5)
add_optimization_path(fig, p_adagrad)
cfig.add_trace(fig.data[0], row=2, col=1)
cfig.add_trace(fig.data[1], row=2, col=1)

# GAF
p_gaf = optimize_function(cone, pi, true_params, noise_params, steps=50, learning_rate=0.25, method='lamb', seed=seeds[3])
fig = plot_true_function(x, y, lambda x: cone(x, true_params))
fig.update_layout(title=f'LARS', title_x=0.5)
add_optimization_path(fig, p_gaf)
cfig.add_trace(fig.data[0], row=2, col=2)
cfig.add_trace(fig.data[1], row=2, col=2)

cfig.update_layout(height=1000, width=1000, title_text='Optimization Algorithms withou GAF', title_x=0.5)
cfig.show()


In [39]:
cfig = make_subplots(rows=2, cols=2, subplot_titles=['SGD', 'Adam', 'AdaGrad', 'LAMB'])

# SGD
p_sgd = optimize_function(cone, pi, true_params, noise_params, steps=50, learning_rate=0.25, batch_size=3, method='sgd', gaf_params=gaf_parameters, seed=seeds[0])
fig = plot_true_function(x, y, lambda x: cone(x, true_params))
fig.update_layout(title=f'SGD', title_x=0.5)
add_optimization_path(fig, p_sgd)
cfig.add_trace(fig.data[0], row=1, col=1)
cfig.add_trace(fig.data[1], row=1, col=1)

# Adam
p_adam = optimize_function(cone, pi, true_params, noise_params, steps=50, learning_rate=0.2, batch_size=10, method='adam', gaf_params=gaf_parameters, seed=seeds[1])
fig = plot_true_function(x, y, lambda x: cone(x, true_params))
fig.update_layout(title=f'Adam', title_x=0.5)
add_optimization_path(fig, p_adam)
cfig.add_trace(fig.data[0], row=1, col=2)
cfig.add_trace(fig.data[1], row=1, col=2)

# AdaGrad
p_adagrad = optimize_function(cone, pi, true_params, noise_params, steps=50, learning_rate=1.0, batch_size=10, method='adagrad', gaf_params=gaf_parameters, seed=seeds[2])
fig = plot_true_function(x, y, lambda x: cone(x, true_params))
fig.update_layout(title=f'AdaGrad', title_x=0.5)
add_optimization_path(fig, p_adagrad)
cfig.add_trace(fig.data[0], row=2, col=1)
cfig.add_trace(fig.data[1], row=2, col=1)

# GAF
p_gaf = optimize_function(cone, pi, true_params, noise_params, steps=50, learning_rate=0.25, method='lamb', gaf_params=gaf_parameters, seed=seeds[3])
fig = plot_true_function(x, y, lambda x: cone(x, true_params))
fig.update_layout(title=f'RMS Prop', title_x=0.5)
add_optimization_path(fig, p_gaf)
cfig.add_trace(fig.data[0], row=2, col=2)
cfig.add_trace(fig.data[1], row=2, col=2)

cfig.update_layout(height=1000, width=1000, title_text='Optimization Algorithms w/ GAF', title_x=0.5)
cfig.show()
